In [1]:
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define transformations to apply to the dataset
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),  # Randomly crop and resize the image to 32x32
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image pixel values
])

# Download and load the CIFAR-10 training dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create DataLoader for training set
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

# Create DataLoader for validation set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

# Download and load the CIFAR-10 test dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for test set
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:05<00:00, 28575934.81it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
next(iter(test_dataset))[0].shape

torch.Size([3, 32, 32])

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define the CNN model
# class cifar_10(nn.Module):
#     def __init__(self):
#         super(cifar_10, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv2 = nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
#         self.relu3 = nn.ReLU()
#         self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
#         self.relu4 = nn.ReLU()
#         self.pool4 = nn.AdaptiveAvgPool2d(1)

#         self.fc1 = nn.Linear(256, 128)
#         self.relu5 = nn.ReLU()

#         self.fc2 = nn.Linear(128, 10)

#     def forward(self, x):
#         x = self.pool1(self.relu1(self.conv1(x)))
#         x = self.pool2(self.relu2(self.conv2(x)))
#         x = self.pool3(self.relu3(self.conv3(x)))
#         x = self.pool4(self.relu4(self.conv4(x)))
#         # print(x.shape)
#         x = x.view(-1, 256)
#         # print(x.shape)
#         x = self.relu5(self.fc1(x))
#         x = self.fc2(x)
#         return x

# model = cifar_10().to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class cifar_10_modified(nn.Module):
    def __init__(self, num_classes = 10, dropout_prob=0.5):
        super(cifar_10_modified, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.AdaptiveAvgPool2d(1)

        self.fc1 = nn.Linear(256, 128)

        self.dropout1 = nn.Dropout(p=dropout_prob)

        self.relu5 = nn.ReLU()

        self.fc2 = nn.Linear(128, num_classes)

        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.pool1(self.batchnorm1(self.relu1(self.conv1(x))))
        x = self.pool2(self.batchnorm2(self.relu2(self.conv2(x))))
        x = self.pool3(self.batchnorm3(self.relu3(self.conv3(x))))
        x = self.pool4(self.batchnorm4(self.relu4(self.conv4(x))))
        # print(x.shape)
        x = x.view(-1, 256)
        # print(x.shape)
        x = self.relu5(self.fc1(x))
        # x = self.dropout1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

model = cifar_10_modified().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [11]:
# import numpy as np
# model(input)

In [12]:
num_epochs = 50

In [13]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    for images, labels in train_loader:
        # Forward pass
        outputs = model(images.to(device))
        loss = criterion(outputs.to(device), labels.to(device))

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images.to(device))
            loss = criterion(outputs.to(device), labels.to(device))
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()

    # Print statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Training Loss: {loss:.4f}, '
          f'Validation Loss: {val_loss / len(val_loader):.4f}, '
          f'Validation Accuracy: {(100 * correct / total):.2f}%')

Epoch 1/50, Training Loss: 1.8674, Validation Loss: 1.7844, Validation Accuracy: 32.88%
Epoch 2/50, Training Loss: 1.4540, Validation Loss: 1.6044, Validation Accuracy: 40.97%
Epoch 3/50, Training Loss: 1.6227, Validation Loss: 1.5257, Validation Accuracy: 44.24%
Epoch 4/50, Training Loss: 1.5115, Validation Loss: 1.4470, Validation Accuracy: 47.38%
Epoch 5/50, Training Loss: 0.8834, Validation Loss: 1.3585, Validation Accuracy: 51.17%
Epoch 6/50, Training Loss: 1.2820, Validation Loss: 1.3361, Validation Accuracy: 52.15%
Epoch 7/50, Training Loss: 1.1451, Validation Loss: 1.2676, Validation Accuracy: 55.24%
Epoch 8/50, Training Loss: 1.2426, Validation Loss: 1.2557, Validation Accuracy: 55.25%
Epoch 9/50, Training Loss: 0.8871, Validation Loss: 1.1881, Validation Accuracy: 57.63%
Epoch 10/50, Training Loss: 1.1358, Validation Loss: 1.1895, Validation Accuracy: 57.97%
Epoch 11/50, Training Loss: 0.9615, Validation Loss: 1.1971, Validation Accuracy: 58.54%
Epoch 12/50, Training Loss: 0.

In [14]:
torch.save(model, "/content/model_path/cifar_10.pth")

In [20]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.6807
Precision: 0.6845
Recall: 0.6807
F1-Score: 0.6773
